In [2]:
from bs4 import BeautifulSoup
import urllib
from urllib.parse import urljoin
import requests

## 1. Extract at least 10 United Nations press releases containing the word “crisis”. Yes, crisis

In [7]:
def scrape_UNpress(num = 10, keyword = 'crisis'):
    URL = 'https://press.un.org/en'
    page = urllib.request.urlopen(URL)
    soup = BeautifulSoup(page, "html.parser")
    # Find the link for more Latest Press Releases
    links = soup.find_all("div", class_="more-link")
    morepress = str([i.find("a").get('href') for i in links if i.text == 'Latest Press Releases'][0])
    keyword_Links = []
    k = 1
    for i in range(4645):
        #read every page
        morepress = urljoin(morepress, '?page=' + str(i))
        press = urllib.request.urlopen(morepress)
        psoup = BeautifulSoup(press, "html.parser")
        content = psoup.find_all("div", class_="field field--name-field-display-title field--type-string field--label-hidden field__item")
        content.link = [i.find('a').get('href') for i in content]
        for link in content.link:
            article_link = urljoin('https://press.un.org/', link)
            article = urllib.request.urlopen(article_link)
            soup = BeautifulSoup(article, "html.parser")
            article_content = soup.find('div', class_ = 'field field--name-body field--type-text-with-summary field--label-hidden field__item')
            if keyword.lower() in article_content.text.lower():
                keyword_Links.append(article_link)
                print(article_link)
                # generate html in text file
                with open(str('1_'+str(k)+'.txt'), 'w') as f:
                    art_page = urllib.request.urlopen(article_link)
                    art_soup = BeautifulSoup(art_page, "html.parser")
                    f.write(str(art_soup))
                k += 1
            if len(keyword_Links) == num:
                return

scrape_UNpress()      

https://press.un.org/en/2023/sgsm21982.doc.htm
https://press.un.org/en/2023/sgsm21980.doc.htm
https://press.un.org/en/2023/sgsm21978.doc.htm
https://press.un.org/en/2023/sgsm21967.doc.htm
https://press.un.org/en/2023/sga2230.doc.htm
https://press.un.org/en/2023/dsgsm1877.doc.htm
https://press.un.org/en/2023/sgsm21959.doc.htm
https://press.un.org/en/2023/sgsm21956.doc.htm
https://press.un.org/en/2023/sgsm21952.doc.htm
https://press.un.org/en/2023/sgsm21951.doc.htm


## 2. Crawl the press room of the European Parliament and extract at least 10 press releases that cover the plenary sessions and contain the word “crisis”.

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains

In [9]:
def EUParliament(URL = 'https://www.europarl.europa.eu/news/en/press-room', num = 10, keyword = 'crisis'):
    '''
    Use selenium to click the "load more button" -- generate more URLs
    Use bs4 for all other steps
    '''
    options = webdriver.ChromeOptions()  ##choose the chrome web browser
    options.binary_location = "/Applications/Chrome.app/Contents/MacOS/Google Chrome"  ##The binary location of Chrome
    chrome_driver_binary = "/Users/charlesxiong/Downloads/chromedriver-mac-x64/chromedriver"  ## The location of chromedriver: an open source tool for automated testing of webapps across many browsers
    driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)  ##Open the driver
    actions = ActionChains(driver)
    driver.get(URL+'?contentType=plenary')
    sleep(2)
    try:
        accept = driver.find_element_by_xpath('//*[@class="epjs_agree"]').click() #need to accept cookies first
        accept.click()
    except:
        for i in range(5):
            sleep(1)
            loadmore = driver.find_element_by_link_text('Load more')
            loadmore.click()
            
    #Use the URLs after clicking load more button n times
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    col = soup.find_all("article", class_="ep_gridcolumn ep-m_product ep-layout_linkmode")
    
    #check if it is a Plenary session - it has to be, because we used "?contentType=plenary"
    links = [i.find('a').get('href') for i in col \
             if 'Plenary session' in i.find('div', class_='ep_subtitle').text]
    t = 0
    for link in links:
        article = urllib.request.urlopen(link)
        soup = BeautifulSoup(article, "html.parser")
        article_content = soup.find(id="website-body").text
        if keyword.lower() in article_content.lower():
            print(link)
            t += 1
            with open(str('2_'+str(t)+'.txt'), 'w') as f:
                art_page = urllib.request.urlopen(link)
                art_soup = BeautifulSoup(art_page, "html.parser")
                f.write(str(art_soup))
            if t == num:
                break

In [10]:
EUParliament()

<ipython-input-9-448d45f17d67>:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)  ##Open the driver


https://www.europarl.europa.eu/news/en/press-room/20211118IPR17619/health-union-medicines-must-be-more-accessible-and-affordable
https://www.europarl.europa.eu/news/en/press-room/20200615IPR81232/parliament-requests-further-action-to-save-eu-tourism
https://www.europarl.europa.eu/news/en/press-room/20200910IPR86823/covid-19-eu-must-step-up-efforts-to-tackle-medicine-shortages
https://www.europarl.europa.eu/news/en/press-room/20140203IPR34618/parliament-pushes-for-enforceable-air-passenger-rights
https://www.europarl.europa.eu/news/en/press-room/20210422IPR02615/civil-protection-faster-eu-response-to-large-scale-emergencies
https://www.europarl.europa.eu/news/en/press-room/20200415IPR77116/covid-19-meps-endorse-enhanced-support-to-eu-fisheries-and-aquaculture
https://www.europarl.europa.eu/news/en/press-room/20201002IPR88432/parliament-demands-a-legally-binding-effective-mechanism-to-protect-eu-values
https://www.europarl.europa.eu/news/en/press-room/20200615IPR81234/covid-19-easing-rul